In [6]:
from datetime import datetime
import pandas as pd   # 추출된 자료를 데이터 프레임 형태로 사용하기 위한 라이브러리 호출 (Call library for cursor to dataframe)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pyhive import presto   # Hive와 연결하기 위한 라이브러리 호출 (Call library for connection to Hive)
from requests.auth import HTTPBasicAuth   # Hive 계정인증을 위한 라이브러리 호출

In [7]:


var_id = 'dopark41'   # Hive ID
var_ps = 'rbqlschlrh23###'   # Hive Password
var_db = 'hive'   # DataBase Name (변경할 필요없음 / No need to change)
var_url = 'cfs-presto.coupang.net'   # DataBase URL (변경할 필요없음 / No need to change)



# Connection Type 2

conn = presto.connect(host=var_url, port=443, protocol='https', catalog=var_db, username=var_id, password=var_ps)



cur = conn.cursor()   # Cursor for Query Sending to DB # 접속 커서를 만드는 코드


### Alternative Connection 
# 구동문


In [8]:
# Test Query
query =\
'''
WITH LOCATIONS AS
(SELECT LOTS.ID
			, LOTS.CENTERID
      , LO.TYPE
			, LOTS.FULLNAME
		  , BINS.NAME
      , BINS.LENGTH
      , BINS.WIDTH
      , BINS.HEIGHT
      , BINS.VOLUME AS CBM
      , CAST(LO.FLOOR AS DOUBLE) FLOOR -- SQL == DOUBLE(실수형), PYTHON == PLOAT(실수형)
      , LO.ZONE
      , CAST(LO.AISLE AS DOUBLE) AISLE
      , CAST(LO.BAY AS DOUBLE) BAY
			, LOTS.SHELF AS BIN
      , LOTS.TEMP -- 임시여부
      , LOTS.ISUSABLE AS ACTIVE_BIN --BIN사용여부 '1 == TRUE(활성화), 0 == FALSE (비활성화)'
FROM ODS.LOTS LOTS
JOIN ODS.CENTER_BINS BINS ON LOTS.BINID = BINS.ID
JOIN ODS.LOCATIONS LO ON LOTS.LOCATIONID = LO.ID
	WHERE 1 = 1
		AND LOTS.CENTERID = 129 -- 센터코드
		AND LO.TYPE IN ('BUFFER','PICKING')
),
INVENTORYS AS
(SELECT INV.LOTID, SKU.EXTERNALID, SKU.NAME, INV.QUANTITY, INV.CREATEDAT
FROM ODS.INVENTORIES INV JOIN ODS.SKUS SKU ON INV.SKUID = SKU.ID
 	WHERE 1 = 1
 		AND QUANTITY > 0)
SELECT  LOC.CENTERID
      , LOC.TYPE
      , LOC.FULLNAME
      , LOC.NAME
      , LOC.LENGTH
      , LOC.WIDTH
      , LOC.HEIGHT
      , LOC.CBM
      , LOC.FLOOR
      , LOC.ZONE
      , LOC.AISLE
      , LOC.BAY
      , LOC.BIN
      , LOC.TEMP
      , LOC.ACTIVE_BIN
			, INV.EXTERNALID
			, INV.NAME
			, INV.QUANTITY
			, INV.CREATEDAT -- 진열 시점
FROM LOCATIONS LOC LEFT JOIN INVENTORYS INV ON LOC.ID = INV.LOTID
ORDER BY 9, 10, 11, 12, 13
'''

cur.execute(query)   # Query를 실행 (Execute query)
df = pd.DataFrame(cur.fetchall(), columns=[ix[0] for ix in cur.description])  # 실행된 Query에서 추출된 자료를 DataFrame형태로 전달 (Send executed query to DataFrame)

conn.close()   # Close DataBase connection (추출완료 후, 연결종료를 실행해야 다른 프로그램에서 DataBase 접속이 가능해집니다) # 접속 종료 코드

In [9]:
# currunt date 구성
now = datetime.now()
current_time = now.strftime("%Y-%m-%d-%H-%M")

In [10]:
# Excel파일로 저장
df.to_excel(f'actual_{current_time}.xlsx')

IllegalCharacterError: 